In [ ]:
import sys
from tqdm import tqdm
import subprocess

def install_with_progress(package_file):
    print(f"📦 Обновляем зависимости из {package_file}...")
    
    try:
        with open(package_file, 'r') as f:
            packages = [line.strip() for line in f if line.strip() and not line.startswith('#')]
    except FileNotFoundError:
        print(f"❌ Файл {package_file} не найден. Пропускаем...")
        return
    
    print(f"🔍 Найдено {len(packages)} пакетов для обновления")
    
    for package in tqdm(packages, desc="📥 Обновление пакетов"):
        try:
            subprocess.run([sys.executable, "-m", "pip", "install", "--upgrade", package, "-q"], check=True, capture_output=True)
        except subprocess.CalledProcessError as e:
            print(f"⚠️ Ошибка при обновлении {package}: {e}")
    
    print("✅ Обновление завершено!")

install_with_progress("requirements.txt")


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import sys
import threading
import select
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
import json
import os
import gc
import numpy as np
import soundfile as sf
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.amp import autocast, GradScaler
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2Model
)
from transformers.utils.quantization_config import BitsAndBytesConfig
from transformers.models.gemma3.configuration_gemma3 import Gemma3TextConfig
from transformers.models.gemma3.modeling_gemma3 import Gemma3ForCausalLM
from huggingface_hub import login
import jiwer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
from huggingface_hub import notebook_login
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from datetime import datetime
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR
import zipfile
import io
import wandb
import glob
import re
import random
import itertools
from IPython.display import Audio, display

def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🌱 Random Seed установлен: {seed}")

set_seed(42)

In [ ]:
# 🔥 Экстренная очистка GPU памяти
print("🧹 Очищаем GPU память...")

# Очищаем кэш PyTorch
torch.cuda.empty_cache()

# Принудительная сборка мусора
gc.collect()

# Очищаем все переменные из памяти
if 'gemma_model' in globals():
    del gemma_model
if 'wav2vec2' in globals():
    del wav2vec2
if 'projector' in globals():
    del projector
if 'train_loader' in globals():
    del train_loader
if 'val_loader' in globals():
    del val_loader

torch.cuda.empty_cache()
gc.collect()

# Проверяем текущее состояние
if torch.cuda.is_available():
    gpu_memory = torch.cuda.memory_allocated() / 1024**3
    print(f"📊 Память после очистки: {gpu_memory:.2f}GB")
    
    # Сброс статистики памяти
    torch.cuda.reset_peak_memory_stats()
    print("✅ Статистика памяти сброшена")
else:
    print("❌ CUDA недоступна")

In [ ]:
def load_checkpoint(path, projector, optimizer, scheduler, device, batch_size):
    global best_val_loss
    print(f"🔄 Загрузка чекпоинта: {path}")
    checkpoint = torch.load(path, map_location=device)
    
    projector.load_state_dict(checkpoint['projector_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    
    start_epoch = checkpoint['epoch']
    saved_step = checkpoint['step']
    best_val_loss = checkpoint.get('best_val_loss', float('inf'))
    
    prev_batch_size = checkpoint['config'].get('batch_size', batch_size)
    
    if prev_batch_size != batch_size:
        print(f"⚠️ Несовпадение batch size: сохранен {prev_batch_size}, текущий {batch_size}")
        
        total_samples_seen = saved_step * prev_batch_size
        adjusted_step = total_samples_seen // batch_size
        
        print(f"📊 Пересчет шагов:")
        print(f"   Сохраненный шаг: {saved_step} (batch_size={prev_batch_size})")
        print(f"   Общее количество образцов: {total_samples_seen:,}")
        print(f"   Скорректированный шаг: {adjusted_step} (batch_size={batch_size})")
        print(f"   Коэффициент: {batch_size/prev_batch_size:.2f}x")
        
        if total_samples_seen % batch_size != 0:
            remaining_samples = total_samples_seen % batch_size
            print(f"   ⚠️ Остаток: {remaining_samples} образцов (будут пропущены)")
        
        global_step = adjusted_step
    else:
        global_step = saved_step
        print(f"✅ Batch size совпадает: {batch_size}")
    
    print(f"✅ Возобновление с эпохи {start_epoch}, шаг {global_step}. Лучший val_loss: {best_val_loss:.4f}")
    return start_epoch, global_step


In [ ]:
# Инициализируем глобальные переменные
best_val_loss = float('inf')
best_checkpoint_path = None
latest_checkpoint_path = None
interactive_mode = True
skip_validation = False

# Переменные которые будут определены позже в обучении
batch_size = None
device = None
learning_rate = None
save_every_steps = None
train_loader = None
val_dataset = None
val_data = None
gemma_model = None
projector = None
wav2vec2 = None
tokenizer = None
prefix_embeds = None
compression_rate_k = None

notebook_login()

In [ ]:
class AudioProjector(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        
        print(f"🔧 AudioProjector (из статьи Llama-AVSR): {input_dim} → 1024 → {output_dim}")
        
        # Точная архитектура из статьи "Large Language Models are Strong Audio-Visual Speech Recognition Learners"
        self.proj = nn.Sequential(
            nn.LayerNorm(input_dim),
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, output_dim),
            nn.LayerNorm(output_dim)
        )
        
        # Инициализация весов
        for layer in self.proj:
            if isinstance(layer, nn.Linear):
                nn.init.xavier_uniform_(layer.weight)
                nn.init.zeros_(layer.bias)
    
    def forward(self, x):
        original_dtype = x.dtype
        x_fp32 = x.to(torch.float32)
        if next(self.proj.parameters()).dtype != torch.float32:
            self.proj = self.proj.float()
        output_fp32 = self.proj(x_fp32)
        return output_fp32.to(original_dtype)


In [ ]:
class TrainingLogger:
    def __init__(self, experiment_name, save_dir):
        self.experiment_name = experiment_name
        self.save_dir = save_dir
        self.logs = {
            'step': [],
            'train_loss': [],
            'val_loss': [],
            'val_perplexity': [],
            'val_wer': [],
            'val_bleu': [],
            'val_rouge_l': [],
            'learning_rate': [],
            'grad_norm': []
        }
        
    def log_step(self, step, train_loss, lr, grad_norm=None):
        wandb.log({
            'train/loss': train_loss,
            'train/learning_rate': lr,
            'train/grad_norm': grad_norm if grad_norm else 0,
            'step': step
        })
        
    def log_validation(self, step, val_metrics):
        self.logs['step'].append(step)
        self.logs['val_loss'].append(val_metrics['loss'])
        self.logs['val_perplexity'].append(val_metrics['perplexity'])
        self.logs['val_wer'].append(val_metrics['wer'])
        self.logs['val_bleu'].append(val_metrics['bleu'])
        self.logs['val_rouge_l'].append(val_metrics['rouge_l'])
        
        wandb.log({
            'val/loss': val_metrics['loss'],
            'val/perplexity': val_metrics['perplexity'],
            'val/wer': val_metrics['wer'],
            'val/bleu': val_metrics['bleu'],
            'val/rouge_l': val_metrics['rouge_l'],
            'step': step
        })
    
    def plot_training_curves(self):
        if len(self.logs['step']) == 0:
            return
            
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle(f'Training Progress: {self.experiment_name}', fontsize=16, fontweight='bold')
        
        axes[0, 0].plot(self.logs['step'], self.logs['val_loss'], 'r-', label='Val Loss', linewidth=2)
        axes[0, 0].set_xlabel('Step')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title('Validation Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
        
        axes[0, 1].plot(self.logs['step'], self.logs['val_perplexity'], 'g-', linewidth=2)
        axes[0, 1].set_xlabel('Step')
        axes[0, 1].set_ylabel('Perplexity')
        axes[0, 1].set_title('Validation Perplexity')
        axes[0, 1].grid(True, alpha=0.3)
        
        axes[0, 2].plot(self.logs['step'], self.logs['val_wer'], 'orange', linewidth=2)
        axes[0, 2].set_xlabel('Step')
        axes[0, 2].set_ylabel('WER')
        axes[0, 2].set_title('Word Error Rate')
        axes[0, 2].grid(True, alpha=0.3)
        
        axes[1, 0].plot(self.logs['step'], self.logs['val_bleu'], 'purple', linewidth=2)
        axes[1, 0].set_xlabel('Step')
        axes[1, 0].set_ylabel('BLEU Score')
        axes[1, 0].set_title('BLEU Score')
        axes[1, 0].grid(True, alpha=0.3)
        
        axes[1, 1].plot(self.logs['step'], self.logs['val_rouge_l'], 'brown', linewidth=2)
        axes[1, 1].set_xlabel('Step')
        axes[1, 1].set_ylabel('ROUGE-L')
        axes[1, 1].set_title('ROUGE-L Score')
        axes[1, 1].grid(True, alpha=0.3)
        
        axes[1, 2].axis('off')
        
        plt.tight_layout()
        
        plot_path = os.path.join(self.save_dir, f'training_curves_step_{self.logs["step"][-1]}.png')
        plt.savefig(plot_path, dpi=300, bbox_inches='tight')
        print(f"📊 График сохранен: {plot_path}")
        plt.show()
    
    def save_logs(self):
        if len(self.logs['step']) == 0:
            return None
        df = pd.DataFrame(self.logs)
        csv_path = os.path.join(self.save_dir, 'validation_logs.csv')
        df.to_csv(csv_path, index=False)
        print(f"📝 Логи валидации сохранены: {csv_path}")
        return df


In [ ]:
class AudioTextDataset(Dataset):
    def __init__(self, data, tokenizer, feature_extractor, zip_path=None):
        self.data = data
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.zip_file = None
        self.zip_manifest = None
        
        if zip_path and os.path.exists(zip_path):
            try:
                self.zip_file = zipfile.ZipFile(zip_path, 'r')
                print(f"📦 ZIP-файл открыт: {zip_path}")
                
                print("⚡️ Создание манифеста ZIP-файла для ускорения доступа...")
                self.zip_manifest = {
                    p: p
                    for p in self.zip_file.namelist()
                    if p.lower().endswith(('.flac', '.wav', '.mp3'))
                }
                print(f"✅ Манифест создан: {len(self.zip_manifest)} аудиофайлов.")

            except Exception as e:
                print(f"⚠️ Ошибка открытия или чтения ZIP: {e}")
                self.zip_file = None
        else:
            print(f"⚠️ ZIP файл не найден: {zip_path}")
            
        print(f"📊 Датасет содержит: {len(self.data)} записей")
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        audio_path = item["audio_path"]
        speaker_text = item["speaker_text"]
        
        try:
            if self.zip_file and self.zip_manifest is not None:
                found_path = self.zip_manifest.get(audio_path)
                if found_path:
                    with self.zip_file.open(found_path) as audio_file:
                        audio_data = audio_file.read()
                        waveform, sr = torchaudio.load(io.BytesIO(audio_data))
                else:
                    raise FileNotFoundError(f"Файл '{audio_path}' не найден в манифесте ZIP.")
            else:
                waveform, sr = torchaudio.load(audio_path)
                
        except Exception as e:
            print(f"⚠️ Ошибка загрузки {audio_path}: {e}")
            waveform = torch.zeros(1, 16000)
            sr = 16000
        
        if sr != self.feature_extractor.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, self.feature_extractor.sampling_rate)
        
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        inputs = self.feature_extractor(
            waveform.squeeze().numpy(),
            sampling_rate=self.feature_extractor.sampling_rate,
            return_tensors="pt"
        )
        
        tokens = self.tokenizer(
            speaker_text,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        )
        return {
            "input_values": inputs.input_values.squeeze(0),
            "input_ids": tokens.input_ids.squeeze(0),
            "attention_mask": tokens.attention_mask.squeeze(0)
        }
    
    def __del__(self):
        if hasattr(self, 'zip_file') and self.zip_file:
            self.zip_file.close()


In [ ]:
def compress_audio_features(audio_features, compression_rate_k):
    """
    Сжимает аудио-признаки путем конкатенации K последовательных векторов.
    Основано на статье "Large Language Models are Strong Audio-Visual Speech Recognition Learners"
    
    Args:
        audio_features: [batch_size, seq_len, hidden_dim]
        compression_rate_k: количество последовательных векторов для объединения
    
    Returns:
        compressed_features: [batch_size, seq_len // K, hidden_dim * K]
    """
    batch_size, seq_len, hidden_dim = audio_features.shape
    
    # Обрезаем последовательность так, чтобы она была кратна K
    new_seq_len = (seq_len // compression_rate_k) * compression_rate_k
    audio_features = audio_features[:, :new_seq_len, :]
    
    # Изменяем форму: [batch_size, seq_len // K, K, hidden_dim]
    reshaped = audio_features.view(batch_size, new_seq_len // compression_rate_k, compression_rate_k, hidden_dim)
    
    # Конкатенируем по последней размерности: [batch_size, seq_len // K, K * hidden_dim]
    compressed = reshaped.view(batch_size, new_seq_len // compression_rate_k, compression_rate_k * hidden_dim)
    
    # Выводим отладочную информацию только для первого вызова
    if hasattr(compress_audio_features, '_first_call'):
        print(f"🗜️ Сжатие аудио: {audio_features.shape} → {compressed.shape} (K={compression_rate_k})")
    else:
        compress_audio_features._first_call = True
        print(f"🗜️ Сжатие аудио: {audio_features.shape} → {compressed.shape} (K={compression_rate_k})")
        print(f"   📊 Исходная длина: {seq_len} → Обрезанная: {new_seq_len} → Сжатая: {new_seq_len // compression_rate_k}")
        print(f"   🔧 Размерность: {hidden_dim} → {compression_rate_k * hidden_dim}")
    
    return compressed


In [ ]:
def collate_fn(batch):
    input_values = [item['input_values'] for item in batch]
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    input_values = pad_sequence(input_values, batch_first=True)
    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=-100)
    attention_mask = pad_sequence(attention_mask, batch_first=True, padding_value=0)
    return {
        'input_values': input_values,
        'input_ids': input_ids,
        'attention_mask': attention_mask
    }

def process_batch(batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k):
    input_values = batch["input_values"].to(device, dtype=torch.bfloat16)
    input_ids = batch["input_ids"].to(device)

    with autocast('cuda' if torch.cuda.is_available() else 'cpu'):
        # Извлекаем последовательность аудио-признаков (НЕ усредняем!)
        audio_embeds = wav2vec2(input_values).last_hidden_state  # [batch_size, seq_len, 768]
        
        # Применяем сжатие по методу из статьи Llama-AVSR
        compressed_audio = compress_audio_features(audio_embeds, compression_rate_k)  # [batch_size, seq_len//K, 768*K]
        
        # Проектируем сжатые признаки
        projected_audio = projector(compressed_audio)  # [batch_size, seq_len//K, output_dim]
        
        # Расширяем префикс для каждого примера в батче
        batch_prefix_embeds = prefix_embeds.expand(projected_audio.size(0), -1, -1)  # [batch_size, prefix_len, output_dim]
        
        # Объединяем префикс и аудио-токены
        prompt_embeds = torch.cat([batch_prefix_embeds, projected_audio], dim=1)  # [batch_size, prefix_len + seq_len//K, output_dim]
        
        # Подготавливаем целевые эмбеддинги для текста
        embedding_input_ids = input_ids.clone()
        embedding_input_ids[embedding_input_ids == -100] = tokenizer.pad_token_id
        target_embeds = model.get_input_embeddings()(embedding_input_ids)

        # Объединяем промпт и целевой текст
        inputs_embeds = torch.cat([prompt_embeds, target_embeds], dim=1)
        
        # Создаем метки: промпт игнорируется (-100), текст учитывается
        prompt_len = prompt_embeds.shape[1]
        prompt_labels = torch.full((projected_audio.size(0), prompt_len), -100, device=device, dtype=torch.long)
        labels = torch.cat([prompt_labels, input_ids], dim=1)

        outputs = model(inputs_embeds=inputs_embeds, labels=labels)
        
    return outputs, prompt_embeds


In [ ]:
def evaluate_with_metrics(model, projector, wav2vec2, dataloader, tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k, beam_width, temperature, top_k, top_p):
    model.eval()
    projector.eval()
    wav2vec2.eval()
    total_loss = 0.0
    total_wer = 0.0
    total_bleu = 0.0
    total_rouge_1 = 0.0
    total_rouge_2 = 0.0
    total_rouge_l = 0.0
    count = 0
    smooth = SmoothingFunction().method1
    rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="🔍 Validation"):
            input_ids = batch["input_ids"].to(device)

            outputs, prompt_embeds = process_batch(
                batch, model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k
            )
            loss = outputs.loss
            total_loss += loss.item()

            with autocast('cuda' if torch.cuda.is_available() else 'cpu'):
                # 🔍 Из статьи: "beam search with a beam width of 15 and temperature of 0.6"
                generated_ids = model.generate(
                    inputs_embeds=prompt_embeds,
                    max_new_tokens=max_new_tokens,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id,
                    num_beams=beam_width,      # 🔍 Beam search вместо greedy
                    temperature=temperature,   # 🌡️ Контролируем случайность
                    do_sample=True,           # 🎲 Включаем sampling для работы с temperature
                    top_k=top_k,              # 📊 Дополнительная фильтрация (параметр)
                    top_p=top_p,              # 📊 Nucleus sampling (параметр)
                    early_stopping=True       # ⏹️ Останавливаемся при EOS
                )
            
            # ‼️ ИСПРАВЛЕНИЕ: При использовании inputs_embeds, generate() возвращает ТОЛЬКО новые токены
            # Неправильно было: generated_ids[:, input_len:] - пытались отрезать промпт, которого там нет
            generated_ids_only = generated_ids

            for j in range(generated_ids.size(0)):
                pred_text = tokenizer.decode(generated_ids_only[j], skip_special_tokens=True).strip()
                ref_text_ids = input_ids[j]
                ref_text_ids = ref_text_ids[ref_text_ids != -100]
                ref_text = tokenizer.decode(ref_text_ids, skip_special_tokens=True).strip()
                
                
                if ref_text and pred_text:
                    current_wer = jiwer.wer(ref_text, pred_text)
                    current_bleu = sentence_bleu([ref_text.split()], pred_text.split(), smoothing_function=smooth)
                    rouge_scores = rouge_scorer_obj.score(ref_text, pred_text)
                    
                    total_wer += current_wer  # type: ignore
                    total_bleu += current_bleu  # type: ignore
                    total_rouge_1 += rouge_scores['rouge1'].fmeasure  # type: ignore
                    total_rouge_2 += rouge_scores['rouge2'].fmeasure  # type: ignore
                    total_rouge_l += rouge_scores['rougeL'].fmeasure  # type: ignore
                    count += 1
                    
    avg_loss = total_loss / len(dataloader)
    perplexity = torch.exp(torch.tensor(avg_loss)).item()
    avg_wer = total_wer / count if count > 0 else 0.0
    avg_bleu = total_bleu / count if count > 0 else 0.0
    avg_rouge_1 = total_rouge_1 / count if count > 0 else 0.0
    avg_rouge_2 = total_rouge_2 / count if count > 0 else 0.0
    avg_rouge_l = total_rouge_l / count if count > 0 else 0.0
    
    print(f"\n📊 Результаты валидации ({count} примеров):")
    print(f"   📉 Loss: {avg_loss:.4f}")
    print(f"   🎯 WER: {avg_wer:.4f} (это доля, не %) = {avg_wer*100:.1f}% ошибок")
    print(f"   📝 BLEU: {avg_bleu:.4f}")
    print(f"   🔍 ROUGE-1: {avg_rouge_1:.4f}")
    
    return {
        'loss': avg_loss, 'perplexity': perplexity,
        'wer': avg_wer, 'bleu': avg_bleu,
        'rouge_1': avg_rouge_1, 'rouge_2': avg_rouge_2, 'rouge_l': avg_rouge_l
    }


In [ ]:
device = torch.device("cuda")
model_id = "google/gemma-3-4b-pt"
audio_model_name = "facebook/wav2vec2-base"

batch_size = 4
num_epochs = 10
learning_rate = 1e-3  # 📈 Из статьи: "learning rate is set to 1e-3 for ASR tasks"
weight_decay = 0.1    # 📈 Из статьи: "weight decay set to 0.1"
max_grad_norm = 5.0
warmup_steps = 100
save_every_steps = 200
save_latest_every_steps = 50
max_new_tokens = 30   # 📉 Уменьшено для предотвращения избыточной генерации (было 50)
compression_rate_k = 3  # 📊 Из статьи: "compression rate K of 3 for the settings"
beam_width = 15       # 🔍 Из статьи: "beam search with a beam width of 15"
temperature = 0.6     # 🌡️ Из статьи: "temperature of 0.6"
top_k = 50           # 📊 Дополнительная фильтрация токенов
top_p = 0.9          # 📊 Nucleus sampling
val_subset_size = 15  # 🧪 Размер выборки для валидации (ускорение)

input_dim = 768  # Wav2Vec2 base hidden size
output_dim = 2560  # Gemma-3 hidden size

experiment_name = f"audio_projector_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
checkpoint_dir = "/home/jovyan/persistent_volume/checkpoints/"
resume_training = True
skip_validation = False
interactive_mode = True

wandb.init(
    project="audio-projector",
    name=experiment_name,
    config={
        "batch_size": batch_size,
        "num_epochs": num_epochs,
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "max_grad_norm": max_grad_norm,
        "max_new_tokens": max_new_tokens,
        "compression_rate_k": compression_rate_k,
        "beam_width": beam_width,
        "temperature": temperature,
        "top_k": top_k,
        "top_p": top_p,
        "val_subset_size": val_subset_size,
        "input_dim": input_dim,
        "output_dim": output_dim,
        "model_id": model_id,
        "audio_model_name": audio_model_name,
        "resume_training": resume_training
    }
)

best_val_loss = float('inf')
best_checkpoint_path = None
latest_checkpoint_path = None

plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True
)

hf_token = os.getenv('HF_TOKEN')
if hf_token:
    login(token=hf_token)

print(f"🚀 Эксперимент: {experiment_name}")
print(f"📁 Чекпоинты: {checkpoint_dir}")
print(f"🖥️  Устройство: {device}")
print(f"⚙️  Конфигурация (параметры из статьи Llama-AVSR):")
print(f"   - Batch size: {batch_size}")
print(f"   - Epochs: {num_epochs} (статья: 10 epochs)")
print(f"   - Learning rate: {learning_rate} (статья: 1e-3 for ASR)")
print(f"   - Weight decay: {weight_decay} (статья: 0.1)")
print(f"   - Gradient clipping: {max_grad_norm}")
print(f"   - Max new tokens: {max_new_tokens} (уменьшено для стабильности)")
print(f"   - Compression rate K: {compression_rate_k} (статья: K=3)")
print(f"   - Beam search width: {beam_width} (статья: 15)")
print(f"   - Temperature: {temperature} (статья: 0.6)")
print(f"   - Top-K: {top_k} (nucleus sampling)")
print(f"   - Top-P: {top_p} (nucleus sampling)")
print(f"   - Val subset size: {val_subset_size} (ускорение валидации)")
print(f"   - Save best every: {save_every_steps} steps")
print(f"   - Save latest every: {save_latest_every_steps} steps")
print(f"   - Resume training: {resume_training}")
print(f"🎵 Audio model: {audio_model_name}")
print(f"🤖 LLM: {model_id}")
print(f"🔗 Projector: {input_dim * compression_rate_k} -> {output_dim} (сжатие K={compression_rate_k}, как в статье)")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("🔧 Применяем патч для загрузки gemma-3-4b-pt как текстовой модели...")
multi_cfg = AutoConfig.from_pretrained(model_id, token=hf_token)

text_cfg_dict = multi_cfg.text_config.to_dict()
text_cfg_dict["vocab_size"] = 262208
text_cfg_dict.update({"bos_token_id": tokenizer.bos_token_id,
                      "eos_token_id": tokenizer.eos_token_id,
                      "pad_token_id": tokenizer.pad_token_id})

text_cfg = Gemma3TextConfig(**text_cfg_dict)
gemma_model = Gemma3ForCausalLM.from_pretrained(
    model_id,
    config=text_cfg,
    torch_dtype=torch.bfloat16,
    quantization_config=quantization_config,
    device_map="auto",
    token=hf_token
)
print("✅ Патч успешно применен, модель загружена.")

gemma_model.eval()
for param in gemma_model.parameters():
    param.requires_grad = False

print(f"Gemma parameters: {sum(p.numel() for p in gemma_model.parameters()):,}")


In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(audio_model_name)
wav2vec2 = Wav2Vec2Model.from_pretrained(audio_model_name)
wav2vec2 = wav2vec2.to(torch.bfloat16).to(device)
wav2vec2.eval()
for param in wav2vec2.parameters():
    param.requires_grad = False

print(f"Wav2vec2 parameters: {sum(p.numel() for p in wav2vec2.parameters()):,}")


In [ ]:
projector_input_dim = input_dim * compression_rate_k  # 768 * 4 = 3072
projector = AudioProjector(projector_input_dim, output_dim).to(device).float()
print(f"🚀 Создан AudioProjector из статьи Llama-AVSR:")
print(f"   ✅ ReLU активация (как в оригинальной статье)")
print(f"   ✅ Двухслойная архитектура: {projector_input_dim} → 1024 → {output_dim}")
print(f"   ✅ LayerNorm на входе и выходе")
print(f"   ✅ Xavier инициализация весов")
print(f"   🗜️ Входная размерность учитывает сжатие K={compression_rate_k}")

optimizer = optim.AdamW(
    projector.parameters(), 
    lr=learning_rate, 
    weight_decay=weight_decay,
    betas=(0.9, 0.999),
    eps=1e-8
)

scheduler = None

scaler = GradScaler()
loss_fn = nn.CrossEntropyLoss(ignore_index=-100)

prefix = "Transcribe speech to text."
prefix_ids = tokenizer(prefix, return_tensors="pt").input_ids.to(device)
with torch.no_grad():
    prefix_embeds = gemma_model.get_input_embeddings()(prefix_ids).to(dtype=torch.bfloat16)

print(f"✅ Оптимизатор: AdamW (lr={learning_rate}, wd={weight_decay})")
print(f"✅ Gradient clipping: {max_grad_norm}")
print(f"✅ Mixed precision: включен")
print(f"✅ Префикс промпта: '{prefix}'")

print(f"\n🔍 Проверка архитектуры проектора (Llama-AVSR):")
for i, layer in enumerate(projector.proj):
    if hasattr(layer, '__class__'):
        layer_name = layer.__class__.__name__
        if layer_name == 'ReLU':
            print(f"   ✅ Слой {i}: {layer_name} (активация ReLU как в статье!)")
        elif 'Linear' in layer_name:
            print(f"   📦 Слой {i}: {layer_name} ({layer.in_features} → {layer.out_features})")
        elif 'LayerNorm' in layer_name:
            print(f"   🔧 Слой {i}: {layer_name} (нормализация)")
        else:
            print(f"   🔧 Слой {i}: {layer_name}")
            
total_params = sum(p.numel() for p in projector.parameters())
print(f"📊 Общее количество параметров проектора: {total_params:,}")


In [ ]:
jsonl_path = "transcripts.jsonl"
zip_path = "LibriSpeech.zip"

resume_batches = 0

print(f"📂 Загружаем данные из {jsonl_path}...")

with open(jsonl_path, "r", encoding="utf-8") as f:
    all_data = [json.loads(line) for line in f]

print(f"📊 Загружено записей: {len(all_data)}")

normalized_data = []
for item in all_data:
    normalized_item = {
        "audio_path": item.get("audio_filepath", ""),
        "speaker_text": item.get("text", ""),
        "language": item.get("language", "en"),
        "source": item.get("source", "unknown")
    }
    normalized_data.append(normalized_item)

total_records = len(normalized_data)
train_data, val_data = train_test_split(normalized_data, test_size=0.1, random_state=42)

if resume_batches > 0:
    skip_samples = resume_batches * batch_size
    train_data = train_data[skip_samples:]
    print(f"🚀 Пропускаем первые {skip_samples} примеров (resume_batches={resume_batches})")

val_subset_data = random.sample(val_data, min(val_subset_size, len(val_data)))

print(f"📊 Размеры данных:")
print(f"   - Train: {len(train_data)} примеров")
print(f"   - Val (полный): {len(val_data)} примеров")
print(f"   - Val (subset): {len(val_subset_data)} примеров")
print(f"   - Ускорение валидации: ~{len(val_data) // len(val_subset_data) if len(val_subset_data) > 0 else 1}x")


In [ ]:
train_dataset = AudioTextDataset(train_data, tokenizer, feature_extractor, zip_path=zip_path)
val_dataset = AudioTextDataset(val_subset_data, tokenizer, feature_extractor, zip_path=zip_path)

resume_step = 0

if resume_step > 0:
    skip_samples = resume_step * batch_size
    original_len = len(train_dataset.data)
    
    if skip_samples < original_len:
        train_dataset.data = train_dataset.data[skip_samples:]
        print(f"🚀 Пропустили {skip_samples} примеров в датасете")
        print(f"   Было: {original_len}, стало: {len(train_dataset.data)}")
    else:
        print(f"⚠️ Пропуск {skip_samples} примеров больше размера датасета {original_len}!")
        print("   Начинаем с начала следующей эпохи")
        resume_step = 0
else:
    print("🌆 Начинаем обучение с начала")

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=False
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=0,
    pin_memory=False
)

print(f"📊 DataLoader готовы: {len(train_loader)} train batches, {len(val_loader)} val batches")


In [ ]:
print("🔧 Переносим все модели на CUDA...")
wav2vec2 = wav2vec2.to(device)
projector = projector.to(device)
gemma_model = gemma_model.to(device)
print("✅ Все модели на CUDA")

print(f"📍 wav2vec2 device: {next(wav2vec2.parameters()).device}")
print(f"📍 projector device: {next(projector.parameters()).device}")
print(f"📍 gemma_model device: {next(gemma_model.parameters()).device}")

total_steps = num_epochs * len(train_loader)
# 📈 Из статьи: "with cosine annealing scheduler"
scheduler = CosineAnnealingLR(
    optimizer, 
    T_max=total_steps,
    eta_min=learning_rate * 0.01  # Минимальный LR = 1% от начального
)

print(f"📅 Общее количество шагов: {total_steps}")
print(f"🔄 Scheduler: CosineAnnealingLR (как в статье)")
print(f"   📈 Начальный LR: {learning_rate}")
print(f"   📉 Минимальный LR: {learning_rate * 0.01}")

os.makedirs(checkpoint_dir, exist_ok=True)
logger = TrainingLogger(experiment_name, checkpoint_dir)


In [ ]:
def find_latest_checkpoint(checkpoint_dir):
    pattern = os.path.join(checkpoint_dir, "latest_checkpoint_bs*_epoch*_step*.pt")    
    checkpoints = glob.glob(pattern) 
    return max(checkpoints, key=os.path.getctime) if checkpoints else None

def find_best_checkpoint(checkpoint_dir):
    pattern = os.path.join(checkpoint_dir, "best_checkpoint_bs*_step*.pt")
    checkpoints = glob.glob(pattern)
    return checkpoints[0] if checkpoints else None

def save_checkpoint(step, epoch, is_best=False):
    global best_checkpoint_path
    
    checkpoint_data = {
        'step': step,
        'epoch': epoch,
        'projector_state_dict': projector.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
        'config': {
            'learning_rate': learning_rate,
            'weight_decay': weight_decay,
            'max_grad_norm': max_grad_norm,
            'batch_size': batch_size,
            'compression_rate_k': compression_rate_k,
            'input_dim': input_dim,
            'output_dim': output_dim,
            'experiment_name': experiment_name
        }
    }
    
    if is_best:
        if best_checkpoint_path and os.path.exists(best_checkpoint_path):
            os.remove(best_checkpoint_path)
        
        best_checkpoint_path = os.path.join(checkpoint_dir, f"best_checkpoint_bs{batch_size}_step_{step}.pt")
        torch.save(checkpoint_data, best_checkpoint_path)
        print(f"🏆 Лучший чекпоинт сохранен: best_checkpoint_bs{batch_size}_step_{step}.pt")
    else:
        checkpoint_path = os.path.join(checkpoint_dir, f"checkpoint_bs{batch_size}_step_{step}.pt")
        torch.save(checkpoint_data, checkpoint_path)
        print(f"💾 Чекпоинт сохранен: checkpoint_bs{batch_size}_step_{step}.pt")

def save_latest_checkpoint(step, epoch):
    global latest_checkpoint_path
    
    checkpoint_data = {
        'step': step,
        'epoch': epoch,
        'projector_state_dict': projector.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict(),
        'best_val_loss': best_val_loss,
        'config': {
            'learning_rate': learning_rate,
            'weight_decay': weight_decay,
            'max_grad_norm': max_grad_norm,
            'batch_size': batch_size,
            'compression_rate_k': compression_rate_k,
            'input_dim': input_dim,
            'output_dim': output_dim,
            'experiment_name': experiment_name
        }
    }
    
    if latest_checkpoint_path and os.path.exists(latest_checkpoint_path):
        os.remove(latest_checkpoint_path)
    
    latest_checkpoint_path = os.path.join(checkpoint_dir, f"latest_checkpoint_bs{batch_size}_epoch_{epoch}_step_{step}.pt")
    torch.save(checkpoint_data, latest_checkpoint_path)
    
    print(f"\r📄 Последний чекпоинт: bs{batch_size}_epoch_{epoch}_step_{step}", end="", flush=True)

def print_vanishing(message):
    print(f"\r{message}", end="", flush=True)

def check_user_input():
    global skip_validation, learning_rate, save_every_steps, interactive_mode, batch_size, train_loader
    
    if not interactive_mode:
        return
        
    try:
        if sys.stdin in select.select([sys.stdin], [], [], 0)[0]:
            user_input = sys.stdin.readline().strip().lower()
            
            if user_input == 's':
                skip_validation = True
                print(f"\n🚫 Валидация будет пропущена на следующем шаге")
            elif user_input == 't':
                print(f"\n🧪 Тестируем модель на случайном примере...")
                try:
                    test_random_sample(val_dataset, val_data, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device)
                except Exception as e:
                    print(f"\n❌ Ошибка тестирования: {e}")
                print(f"\n⏮️ Продолжаем обучение...\n")
            elif user_input == 'm':
                try:
                    gpu_memory = torch.cuda.memory_allocated(device) / 1024**3
                    gpu_memory_max = torch.cuda.max_memory_allocated(device) / 1024**3
                    print(f"\n📊 GPU память: {gpu_memory:.1f}GB / {gpu_memory_max:.1f}GB пик")
                except:
                    print(f"\n❌ Не удалось получить информацию о GPU памяти")
            elif user_input.startswith('bs='):
                try:
                    new_batch_size = int(user_input.split('=')[1])
                    if new_batch_size > 0 and new_batch_size <= 128:
                        old_batch_size = batch_size
                        batch_size = new_batch_size
                        
                        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
                        print(f"\n🔄 Batch size изменен: {old_batch_size} → {new_batch_size}")
                        print(f"📊 Новое количество батчей в эпохе: {len(train_loader)}")
                    else:
                        print(f"\n❌ Batch size должен быть от 1 до 128")
                except:
                    print(f"\n❌ Неверный формат batch size. Используйте: bs=32")
            elif user_input.startswith('lr='):
                try:
                    new_lr = float(user_input.split('=')[1])
                    learning_rate = new_lr
                    for param_group in optimizer.param_groups:
                        param_group['lr'] = new_lr
                    print(f"\n📈 Learning rate изменен на: {new_lr}")
                except:
                    print(f"\n❌ Неверный формат LR. Используйте: lr=0.001")
            elif user_input.startswith('save='):
                try:
                    new_save = int(user_input.split('=')[1])
                    save_every_steps = new_save
                    print(f"\n💾 Интервал сохранения изменен на: {new_save} шагов")
                except:
                    print(f"\n❌ Неверный формат. Используйте: save=100")
            elif user_input == 'help':
                print(f"\n📋 Команды:")
                print(f"   s - пропустить следующую валидацию")
                print(f"   t - протестировать модель на случайном примере")
                print(f"   m - показать использование GPU памяти")
                print(f"   bs=32 - изменить batch size")
                print(f"   lr=0.001 - изменить learning rate")
                print(f"   save=100 - изменить интервал сохранения")
                print(f"   help - показать эту справку")
    except:
        pass

print("🎮 Интерактивный режим включен! Команды:")
print("   s - пропустить валидацию")
print("   t - протестировать модель на случайном примере")
print("   m - показать использование GPU памяти")
print("   bs=32 - изменить batch size")
print("   lr=0.001 - изменить learning rate")  
print("   save=100 - изменить интервал сохранения")
print("   help - справка")


In [ ]:
start_epoch = 0
global_step = 0

if resume_training:
    checkpoint_path = find_latest_checkpoint(checkpoint_dir)
    
    if checkpoint_path:
        checkpoint_epoch, global_step = load_checkpoint(checkpoint_path, projector, optimizer, scheduler, device, batch_size)
        start_epoch = checkpoint_epoch - 1
        print(f"🔄 Продолжаем обучение с шага {global_step}, эпохи {checkpoint_epoch}")
    else:
        print("⚠️ Чекпоинты не найдены. Начинаем новое обучение.")
        resume_training = False

print(f"🚀 Начинаем обучение Audio Projector!")
print(f"📈 W&B проект: {wandb.run.project} / {wandb.run.name}")
print(f"🔄 Режим: {'Возобновление' if resume_training else 'Новое обучение'}")


In [ ]:
for epoch in range(start_epoch, num_epochs):
    print(f"\n{'='*50}")
    print(f"🔄 ЭПОХА {epoch+1}/{num_epochs}")
    print(f"{'='*50}")
    
    projector.train()
    wav2vec2.eval()
    gemma_model.eval()
    
    is_resumed_epoch = resume_training and epoch == start_epoch
    batches_to_skip = (global_step % len(train_loader)) if is_resumed_epoch else 0
    
    if batches_to_skip > 0:
        print(f"⏭️  Пропускаем {batches_to_skip} батчей в эпохе {epoch+1}")
        print(f"⚡ НЕ загружая данные для пропущенных батчей...")

    progress_bar = tqdm(
        enumerate(train_loader), 
        total=len(train_loader),
        initial=batches_to_skip,
        desc=f"Epoch {epoch+1}"
    )

    first_batch_logged = False

    for batch_idx, batch in progress_bar:
        real_batch_number = global_step + batch_idx
        
        if not first_batch_logged:
            print(f"\n✅ Начинаем реальную обработку с батча {batch_idx} (глобальный шаг {real_batch_number})")
            print(f"   Размер аудио-тензора: {batch['input_values'].shape}")
            print(f"   Размер текстового-тензора: {batch['input_ids'].shape}")
            first_batch_logged = True
                
        current_global_step = real_batch_number
            
        optimizer.zero_grad()
        
        outputs, _ = process_batch(
            batch, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device, compression_rate_k
        )
        loss = outputs.loss
        
        # 📊 Мониторинг GPU памяти после первого forward pass
        if current_global_step == 0 or current_global_step % 10 == 0:
            if torch.cuda.is_available():
                gpu_memory = torch.cuda.memory_allocated(device) / 1024**3
                gpu_memory_reserved = torch.cuda.memory_reserved(device) / 1024**3
                gpu_memory_max = torch.cuda.max_memory_allocated(device) / 1024**3
                print(f"\n🖥️  GPU Memory (шаг {current_global_step}):")
                print(f"   📈 Выделено: {gpu_memory:.2f}GB")
                print(f"   📦 Зарезервировано: {gpu_memory_reserved:.2f}GB") 
                print(f"   🔥 Пик использования: {gpu_memory_max:.2f}GB")
                print(f"   🎯 Batch size: {batch_size} для этого объема памяти")
                
                # Рекомендации по batch size на основе использования памяти
                if gpu_memory > 20:
                    print(f"   ⚠️  Высокое потребление памяти! Рекомендуется уменьшить batch_size")
                elif gpu_memory < 8:
                    print(f"   ✅ Можно попробовать увеличить batch_size для лучшего использования GPU")
                else:
                    print(f"   👍 Оптимальное использование GPU памяти")
                
        scaler.scale(loss).backward()
        
        scaler.unscale_(optimizer)
        grad_norm = torch.nn.utils.clip_grad_norm_(projector.parameters(), max_grad_norm)
        
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()
        
        current_lr = scheduler.get_last_lr()[0]
        
        logger.log_step(current_global_step, loss.item(), current_lr, grad_norm.item())
        
        progress_bar.set_postfix({
            'Loss': f'{loss.item():.4f}',
            'LR': f'{current_lr:.2e}',
            'Step': current_global_step
        })
        
        check_user_input()
        
        if current_global_step % save_latest_every_steps == 0:
            save_latest_checkpoint(current_global_step, epoch + 1)
            print_vanishing(f"💾 Сохранен latest checkpoint на шаге {current_global_step}")
        
        if current_global_step % save_every_steps == 0:
            if skip_validation:
                print(f"\n🚫 Валидация пропущена на шаге {current_global_step} (пользователь)")
                skip_validation = False
            else:
                print(f"\n🔍 Валидация на шаге {current_global_step}...")
                
                val_metrics = evaluate_with_metrics(
                    gemma_model, projector, wav2vec2, val_loader, 
                    tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k,
                    beam_width, temperature, top_k, top_p  # 🔍 Все параметры генерации
                )
            
                logger.log_validation(current_global_step, val_metrics)
                
                print(f"📊 Результаты валидации (шаг {current_global_step}):")
                print(f"   Loss: {val_metrics['loss']:.4f}")
                print(f"   Perplexity: {val_metrics['perplexity']:.2f}")
                print(f"   WER: {val_metrics['wer']:.3f}")
                print(f"   BLEU: {val_metrics['bleu']:.3f}")
                print(f"   ROUGE-L: {val_metrics['rouge_l']:.3f}")
                
                is_best = val_metrics['loss'] < best_val_loss
                if is_best:
                    best_val_loss = val_metrics['loss']
                    print(f"🏆 Новый лучший результат! Loss: {best_val_loss:.4f}")
                
                save_checkpoint(current_global_step, epoch + 1, is_best)
                logger.plot_training_curves()
                
                del val_metrics
                torch.cuda.empty_cache()
                
                projector.train()
    
    if is_resumed_epoch:
        resume_training = False
        print(f"✅ Эпоха {epoch+1} завершена, переходим к обычному режиму")


In [ ]:
print(f"\n{'='*50}")
print(f"🎉 ОБУЧЕНИЕ ЗАВЕРШЕНО!")
print(f"{'='*50}")

print("🔍 Финальная валидация на полном validation set...")
print("💡 Это займет больше времени, но даст более точную оценку")

full_val_dataset = AudioTextDataset(val_data, tokenizer, feature_extractor, zip_path=zip_path)
full_val_loader = DataLoader(full_val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

final_val_metrics = evaluate_with_metrics(
    gemma_model, projector, wav2vec2, full_val_loader, 
    tokenizer, prefix_embeds, device, max_new_tokens, compression_rate_k,
    beam_width, temperature, top_k, top_p  # 🔍 Все параметры генерации в финальной валидации
)

print(f"\n📊 Финальные результаты (полный validation set, {len(full_val_dataset)} примеров):")
print(f"   Loss: {final_val_metrics['loss']:.4f}")
print(f"   Perplexity: {final_val_metrics['perplexity']:.2f}")
print(f"   WER: {final_val_metrics['wer']:.3f}")
print(f"   BLEU: {final_val_metrics['bleu']:.3f}")
print(f"   ROUGE-L: {final_val_metrics['rouge_l']:.3f}")

logger.log_validation(global_step, final_val_metrics)

del final_val_metrics, full_val_dataset, full_val_loader
torch.cuda.empty_cache()

logger.plot_training_curves()
final_logs_df = logger.save_logs()

if final_logs_df is not None:
    print(f"\n📋 Финальная статистика:")
    print(final_logs_df.tail().round(4))

final_model_path = os.path.join(checkpoint_dir, "final_projector.pt")
torch.save(projector.state_dict(), final_model_path)
print(f"🏆 Финальная модель: {final_model_path}")

wandb.finish()
print("🏁 wandb завершён корректно.")

torch.cuda.empty_cache()
gc.collect()
print("✨ Оперативная память очищена")


In [ ]:
def test_random_sample(dataset, original_data, model, projector, wav2vec2, tokenizer, prefix_embeds, device, beam_width, temperature, top_k, top_p, max_new_tokens):
    model.eval()
    projector.eval()

    idx = random.randint(0, len(dataset) - 1)
    original_sample_info = original_data[idx]
    
    print(f"--- 🧪 Тестируем модель на примере #{idx} ---")
    print(f"📄 Файл: {original_sample_info['audio_path']}")

    audio_path = original_sample_info['audio_path']
    zip_file = getattr(dataset, 'zip_file', None)
    waveform = None
    sr = 16000

    try:
        if zip_file:
            found_path = next((p for p in zip_file.namelist() if p.endswith(os.path.basename(audio_path))), None)
            if found_path:
                with zip_file.open(found_path) as audio_file:
                    waveform, sr = torchaudio.load(io.BytesIO(audio_file.read()))
            else:
                raise FileNotFoundError(f"Аудио не найдено в ZIP: {audio_path}")
        elif os.path.exists(audio_path):
            waveform, sr = torchaudio.load(audio_path)
        else:
            raise FileNotFoundError(f"Аудио не найдено на диске: {audio_path}")
    except Exception as e:
        print(f"❌ Ошибка загрузки аудио: {e}")
        return

    with torch.no_grad():
        if sr != feature_extractor.sampling_rate:
            waveform = torchaudio.functional.resample(waveform, sr, feature_extractor.sampling_rate)
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        input_values = feature_extractor(waveform.squeeze().numpy(), sampling_rate=feature_extractor.sampling_rate, return_tensors="pt").input_values
        input_values = input_values.to(device, dtype=torch.bfloat16)

        with autocast('cuda'):
            # Применяем новый подход - НЕ усредняем, а сжимаем
            audio_embeds = wav2vec2(input_values).last_hidden_state  # [1, seq_len, 768]
            compressed_audio = compress_audio_features(audio_embeds, compression_rate_k)  # [1, seq_len//K, 768*K]
            projected_audio = projector(compressed_audio)  # [1, seq_len//K, output_dim]
            
            batch_prefix_embeds = prefix_embeds.expand(projected_audio.size(0), -1, -1)
            prompt_embeds = torch.cat([batch_prefix_embeds, projected_audio], dim=1)

            # 🔍 Применяем beam search как в валидации (все параметры)
            generated_ids = model.generate(
                inputs_embeds=prompt_embeds, max_new_tokens=max_new_tokens,
                eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,
                num_beams=beam_width, temperature=temperature,
                do_sample=True, top_k=top_k, top_p=top_p, early_stopping=True
            )
            
    generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True).strip()
    reference_text = original_sample_info['speaker_text']

    print(f"\n🗣️  Оригинальный текст:")
    print(f"    '{reference_text}'")
    print(f"\n🤖  Результат модели:")
    print(f"    '{generated_text}'")
    
    if reference_text and generated_text:
        wer_score = jiwer.wer(reference_text, generated_text)
        print(f"\n📊 WER (Word Error Rate): {wer_score:.3f}")
        if wer_score < 0.3:
            print("✅ Отличный результат!")
        elif wer_score < 0.5:
            print("👍 Хороший результат")
        elif wer_score < 0.8:
            print("⚠️ Средний результат")
        else:
            print("❌ Требует улучшения")
    
    print(f"\n🎵 Воспроизведение аудио ({waveform.shape[1]/sr:.1f} сек):")
    display(Audio(waveform.numpy(), rate=sr))

print("🧪 Для тестирования модели выполните:")
print("test_random_sample(val_dataset, val_data, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device, beam_width, temperature, top_k, top_p, max_new_tokens)")
print("\n🔧 Или загрузите лучший чекпоинт перед тестированием:")
print("# best_path = find_best_checkpoint(checkpoint_dir)")
print("# if best_path: load_checkpoint(best_path, projector, optimizer, scheduler)")
print("# test_random_sample(val_dataset, val_data, gemma_model, projector, wav2vec2, tokenizer, prefix_embeds, device, beam_width, temperature, top_k, top_p, max_new_tokens)")
